In [2]:
import mnist
import numpy as np
from conv import Conv3x3
from maxpool import MaxPool2
from softmax import Softmax

In [ ]:
# We only use the first 1k examples of each set in the interest of time.
train_images = mnist.train_images()[:1000]
train_labels = mnist.train_labels()[:1000]
test_images = mnist.test_images()[:1000]
test_labels = mnist.test_labels()[:1000]

conv = Conv3x3(8)                  # 28x28x1 -> 26x26x8
pool = MaxPool2()                  # 26x26x8 -> 13x13x8
softmax = Softmax(13 * 13 * 8, 10) # 13x13x8 -> 10

In [3]:
def forward(image, label):
  '''
  Completes a forward pass of the CNN and calculates the accuracy and
  cross-entropy loss.
  - image is a 2d numpy array
  - label is a digit
  '''
  # We transform the image from [0, 255] to [-0.5, 0.5] to make it easier
  # to work with. This is standard practice.
  out = conv.forward((image / 255) - 0.5)
  out = pool.forward(out)
  out = softmax.forward(out)

  # Calculate cross-entropy loss and accuracy. np.log() is the natural log.
  loss = -np.log(out[label])
  acc = 1 if np.argmax(out) == label else 0

  return out, loss, acc

def train(im, label, lr=.005):
  '''
  Completes a full training step on the given image and label.
  Returns the cross-entropy loss and accuracy.
  - image is a 2d numpy array
  - label is a digit
  - lr is the learning rate
  '''
  # Forward
  out, loss, acc = forward(im, label)

  # Calculate initial gradient
  gradient = np.zeros(10)
  gradient[label] = -1 / out[label]

  # Backprop
  gradient = softmax.backprop(gradient, lr)
  gradient = pool.backprop(gradient)
  gradient = conv.backprop(gradient, lr)

  return loss, acc

print('MNIST CNN initialized!')

# Train the CNN for 3 epochs
for epoch in range(3):
  print('--- Epoch %d ---' % (epoch + 1))

  # Shuffle the training data
  permutation = np.random.permutation(len(train_images))
  train_images = train_images[permutation]
  train_labels = train_labels[permutation]

  # Train!
  loss = 0
  num_correct = 0
  for i, (im, label) in enumerate(zip(train_images, train_labels)):
    if i % 100 == 99:
      print(
        '[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%' %
        (i + 1, loss / 100, num_correct)
      )
      loss = 0
      num_correct = 0

    l, acc = train(im, label)
    loss += l
    num_correct += acc

# Test the CNN
print('\n--- Testing the CNN ---')
loss = 0
num_correct = 0
for im, label in zip(test_images, test_labels):
  _, l, acc = forward(im, label)
  loss += l
  num_correct += acc

num_tests = len(test_images)
print('Test Loss:', loss / num_tests)
print('Test Accuracy:', num_correct / num_tests)


MNIST CNN initialized!
--- Epoch 1 ---
[Step 100] Past 100 steps: Average Loss 2.167 | Accuracy: 27%
[Step 200] Past 100 steps: Average Loss 1.892 | Accuracy: 42%
[Step 300] Past 100 steps: Average Loss 1.363 | Accuracy: 61%
[Step 400] Past 100 steps: Average Loss 1.112 | Accuracy: 67%
[Step 500] Past 100 steps: Average Loss 0.895 | Accuracy: 74%
[Step 600] Past 100 steps: Average Loss 0.717 | Accuracy: 76%
[Step 700] Past 100 steps: Average Loss 0.736 | Accuracy: 77%
[Step 800] Past 100 steps: Average Loss 0.735 | Accuracy: 79%
[Step 900] Past 100 steps: Average Loss 0.709 | Accuracy: 80%
[Step 1000] Past 100 steps: Average Loss 0.629 | Accuracy: 81%
--- Epoch 2 ---
[Step 100] Past 100 steps: Average Loss 0.412 | Accuracy: 90%
[Step 200] Past 100 steps: Average Loss 0.737 | Accuracy: 79%
[Step 300] Past 100 steps: Average Loss 0.566 | Accuracy: 84%
[Step 400] Past 100 steps: Average Loss 0.449 | Accuracy: 87%
[Step 500] Past 100 steps: Average Loss 0.419 | Accuracy: 89%
[Step 600] Pas